1. Типы шумов. Выбрать произвольное изображение. Получить
искаженные различными шумами изображения с помощью
функции imnoise() с отличными от значений по умолчанию
параметрами.

2. Низкочастотная фильтрация. Обработать полученные в
предыдущем пункте искаженные изображения фильтром
Гаусса и контргармоническим усредняющим фильтром с различными значениями параметра 𝑄.

4. Высокочастотная фильтрация. Выбрать исходное изображение. 
Выделить границы фильтрами 

Робертса, 
Превитта, 
Собела, 
Лапласа,
алгоритмом Кэнни.

**Типы шумов**

Цифровые изображения, полученные различными оптикоэлектронными приборами, могут содержать в себе разнообразные
искажения, обусловленные разного рода помехами, которые принято называть шумом. Шум на изображении затрудняет его 
обработку автоматическими средствами и, поскольку шум может иметь различную природу, для его успешного подавления 
необходимо определить адекватную математическую модель. Рассмотрим наиболее распространенные модели шумов. 

In [2283]:
import cv2
import skimage
import skimage.filters
import skimage.restoration
import scipy
import scipy.ndimage
import scipy.signal
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from numba import njit, jit
from math import *
import warnings
warnings.filterwarnings('ignore')
export_parallel = True
use_graph = False

In [2284]:
# Исходное изображение до наложения шума
image = cv2.cvtColor(cv2.imread("data/night_city.jpg"), cv2.COLOR_BGR2GRAY)
print(image.shape)

images_noised = {}

# Общая для всех зашумлённость
amount = 0.2
var = 0.2
mean = 0.0
lam = 20.0

(2074, 3782)


In [2285]:
# Исходное изображение до наложения шума
if use_graph:
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

**Импульсный шум**

При импульсном шуме сигнал искажается выбросами с очень
большими отрицательными или положительными значениями малой длительностью и может возникать, например, из-за ошибок
декодирования. Такой шум приводит к появлению на изображении белых («соль») или черных («перец») точек, поэтому зачастую
называется точечным шумом. Для его описания следует принять
во внимание тот факт, что появление шумового выброса в каждом
пикселе 𝐼(𝑥,𝑦) не зависит ни от качества исходного изображения, ни
от наличия шума в других точках и имеет вероятность появления
𝑝, причем значение интенсивности пикселя 𝐼(𝑥,𝑦) будет изменено
на значение 𝑑 ∈ [0,255]

In [2286]:
# Наложение импульсного шума
image_noised_s_and_p = skimage.util.random_noise(image, mode="s&p", amount=amount, salt_vs_pepper=0.3)
image_noised_s_and_p = np.clip(image_noised_s_and_p, 0.0, 1.0)
image_noised_s_and_p = skimage.img_as_ubyte(image_noised_s_and_p)
images_noised["salt and pepper"] = image_noised_s_and_p.copy()

image_noised_all = image_noised_s_and_p

# Изображение после наложения импульсного шума
if use_graph:
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_noised_s_and_p, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

**Аддитивный шум**

Аддитивный шум описывается следующим выражением:

𝐼𝑛𝑒𝑤(𝑥,𝑦) = 𝐼(𝑥,𝑦) + 𝜂(𝑥,𝑦),

где 𝐼𝑛𝑒𝑤 — зашумленное изображение, 
𝐼 — исходное изображение,
𝜂 — не зависящий от сигнала аддитивный шум с гауссовым или
любым другим распределением функции плотности вероятности.

**Мультипликативный шум**

Мультипликативный шум описывается следующим выражением:

𝐼𝑛𝑒𝑤(𝑥,𝑦) = 𝐼(𝑥,𝑦) · 𝜂(𝑥,𝑦),

где 𝐼𝑛𝑒𝑤 — зашумленное изображение, 𝐼 — исходное изображение, 𝜂
— не зависящий от сигнала мультипликативный шум, умножающий
зарегистрированный сигнал. В качестве примера можно привести 
зернистость фотопленки, ультразвуковые изображения и т.д. Частным случаем мультипликативного шума является спекл-шум, 
который появляется на изображениях, полученных устройствами с когерентным формированием изображений, например, 
медицинскими
сканерами или радарами. На таких изображениях можно отчетливо
наблюдать светлые пятна, крапинки (спеклы), которые разделены
темными участками изображения.

In [2287]:
# Наложение мультипликативного шума
image_noised_multi = skimage.util.random_noise(image, mode="speckle", var=var, mean=mean)
image_noised_multi = np.clip(image_noised_multi, 0.0, 1.0)
image_noised_multi = skimage.img_as_ubyte(image_noised_multi)
images_noised["speckle"] = image_noised_multi.copy()

image_noised_all = skimage.util.random_noise(image_noised_all, mode="speckle", var=var, mean=mean)
image_noised_all = np.clip(image_noised_all, 0.0, 1.0)
image_noised_all = skimage.img_as_ubyte(image_noised_all)

# Изображение после наложения мультипликативного шума
if use_graph:
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_noised_multi, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

**Гауссов (нормальный) шум**

Гауссов шум на изображении может возникать в следствие недостатка освещенности сцены, высокой температуры и т.д. 
Модель шума широко распространена в задачах низкочастотной фильтрации изображений. 

In [2288]:
# Наложение Гауссова шума
image_noised_gaussian = skimage.util.random_noise(image, mode="gaussian", var=var, mean=mean)
image_noised_gaussian = np.clip(image_noised_gaussian, 0.0, 1.0)
image_noised_gaussian = skimage.img_as_ubyte(image_noised_gaussian)
images_noised["gaussian"] = image_noised_gaussian.copy()

image_noised_all = skimage.util.random_noise(image_noised_all, mode="gaussian", var=var, mean=mean)
image_noised_all = np.clip(image_noised_all, 0.0, 1.0)
image_noised_all = skimage.img_as_ubyte(image_noised_all)

# Изображение после наложения Гауссова шума
if use_graph:
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_noised_gaussian, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

**Шум квантования**

Зависит от выбранного шага квантования и самого сигнала.
Шум квантования может приводить, например, к появлению ложных контуров вокруг объектов или убирать слабо 
контрастные детали на изображении. Такой шум не устраняется. Приближенно
шум квантования можно описать распределением Пуассона

In [2289]:
# Создание шума квантования с заданными параметрами распределения Пуассона
noise_poisson = np.random.poisson(lam=lam, size=image.shape) * (1.0 / 255.0)
noise_poisson = np.clip(noise_poisson, 1.0 / 255.0, 1.0)

# Наложение шума квантования
image_noised_poisson = skimage.util.random_noise(image, mode="localvar", local_vars=noise_poisson)
image_noised_poisson = np.clip(image_noised_poisson, 0.0, 1.0)
image_noised_poisson = skimage.img_as_ubyte(image_noised_poisson)
images_noised["poisson"] = image_noised_poisson.copy()

image_noised_all = skimage.util.random_noise(image_noised_all, mode="localvar", local_vars=noise_poisson)
image_noised_all = np.clip(image_noised_all, 0.0, 1.0)
image_noised_all = skimage.img_as_ubyte(image_noised_all)

# Изображение после наложения шума квантования
if use_graph:
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_noised_poisson, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

In [2290]:
# Изображение после наложения всех шумов
if use_graph:
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_noised_all, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

### 2 Низкочастотная фильтрация

Низкочастотные пространственные фильтры ослабляют высокочастотные компоненты (области с сильным изменением интенсивностей) и оставляют низкочастотные компоненты изображения
без изменений. Используются для снижения уровня шума и удаления высокочастотных компонент, что позволяет повысить точность
исследования содержания низкочастотных компонент. В результате применения низкочастотных фильтров получим сглаженное или
размытое изображение. Главными отличительными особенностями
являются:

1. неотрицательные коэффициенты маски;

2. сумма всех коэффициентов равна единице.

### 2.1 Фильтр Гаусса

In [2291]:
# Применеие фильтра Гаусса ко всем зашумлённым изображениям
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name)
        image_filtered = skimage.filters.gaussian(images_noised[image_name], sigma=4.0)
        image_filtered = np.clip(image_filtered, 0.0, 1.0)
        image_filtered = skimage.img_as_ubyte(image_filtered)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

### 2.2 Контргармонический усредняющий фильтр с различными значениями параметра 𝑄

In [2292]:
def counter_harmonic_mean_filter(image, kernel_size=3, Q=0):
    rows, cols = image.shape
    kernel_mid = kernel_size // 2
    kernel_k = 1 / pow(kernel_size, 2.0)
    
    image_bordered = cv2.copyMakeBorder(
        image,
        top=kernel_size // 2,
        bottom=kernel_size // 2 + kernel_size % 2,
        left=kernel_size // 2,
        right=kernel_size // 2 + kernel_size % 2,
        borderType=cv2.BORDER_REPLICATE,
        value=[mean, mean, mean]
    )
    
    image_float = np.clip(image / 255.0, 0.0, 1.0)
    image_bordered_float = np.zeros_like(image_bordered, dtype=np.float64)
    
    image_float_power = []
    for q in range(2):
        image_float_power.append(np.power(image_float, Q + q))
        image_float_power[q] = np.clip(image_float_power[q], 0.0, 10)
    for y in range(-kernel_mid, kernel_mid + kernel_size % 2):
        for x in range(-kernel_mid, kernel_mid + kernel_size % 2):
            image_bordered_float[kernel_mid + y:kernel_mid + rows + y,
            kernel_mid + x:kernel_mid + cols + x] += kernel_k * image_float_power[1] / image_float_power[0]
    image_bordered_float = np.clip(image_bordered_float, 0.0, 1.0)
    image = image_bordered_float[kernel_mid:rows+kernel_mid, kernel_mid:cols+kernel_mid] * 255.0
    return image.astype(np.uint8)

In [2293]:
kernel_size = 7
Q = -1.5
# Применеие Контргармонического усредняющего фильтра с различными значениями параметра 𝑄 
# ко всем зашумлённым изображениям
print(f"Q = {Q}")
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = counter_harmonic_mean_filter(images_noised[image_name], kernel_size=kernel_size, Q=Q)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

Q = -1.5


In [2294]:
Q = -1.25
# Применеие Контргармонического усредняющего фильтра с различными значениями параметра 𝑄 
# ко всем зашумлённым изображениям
print(f"Q = {Q}")
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = counter_harmonic_mean_filter(images_noised[image_name], kernel_size=kernel_size, Q=Q)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

Q = -1.25


In [2295]:
Q = -1.0
# Применеие Контргармонического усредняющего фильтра с различными значениями параметра 𝑄 
# ко всем зашумлённым изображениям
print(f"Q = {Q}")
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = counter_harmonic_mean_filter(images_noised[image_name], kernel_size=kernel_size, Q=Q)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

Q = -1.0


In [2296]:
Q = -0.75
# Применеие Контргармонического усредняющего фильтра с различными значениями параметра 𝑄 
# ко всем зашумлённым изображениям
print(f"Q = {Q}")
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = counter_harmonic_mean_filter(images_noised[image_name], kernel_size=kernel_size, Q=Q)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

Q = -0.75


In [2297]:
Q = -0.5
# Применеие Контргармонического усредняющего фильтра с различными значениями параметра 𝑄 
# ко всем зашумлённым изображениям
print(f"Q = {Q}")
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = counter_harmonic_mean_filter(images_noised[image_name], kernel_size=kernel_size, Q=Q)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

Q = -0.5


In [2298]:
Q = -0.25
# Применеие Контргармонического усредняющего фильтра с различными значениями параметра 𝑄 
# ко всем зашумлённым изображениям
print(f"Q = {Q}")
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = counter_harmonic_mean_filter(images_noised[image_name], kernel_size=kernel_size, Q=Q)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

Q = -0.25


In [2299]:
Q = 0.0
# Применеие Контргармонического усредняющего фильтра с различными значениями параметра 𝑄 
# ко всем зашумлённым изображениям
print(f"Q = {Q}")
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = counter_harmonic_mean_filter(images_noised[image_name], kernel_size=kernel_size, Q=Q)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

Q = 0.0


In [2300]:
Q = 0.25
# Применеие Контргармонического усредняющего фильтра с различными значениями параметра 𝑄 
# ко всем зашумлённым изображениям
print(f"Q = {Q}")
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = counter_harmonic_mean_filter(images_noised[image_name], kernel_size=kernel_size, Q=Q)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

Q = 0.25


In [2301]:
Q = 0.5
# Применеие Контргармонического усредняющего фильтра с различными значениями параметра 𝑄 
# ко всем зашумлённым изображениям
print(f"Q = {Q}")
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = counter_harmonic_mean_filter(images_noised[image_name], kernel_size=kernel_size, Q=Q)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

Q = 0.5


In [2302]:
Q = 0.75
# Применеие Контргармонического усредняющего фильтра с различными значениями параметра 𝑄 
# ко всем зашумлённым изображениям
print(f"Q = {Q}")
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = counter_harmonic_mean_filter(images_noised[image_name], kernel_size=kernel_size, Q=Q)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

Q = 0.75


In [2303]:
Q = 1.0
# Применеие Контргармонического усредняющего фильтра с различными значениями параметра 𝑄 
# ко всем зашумлённым изображениям
print(f"Q = {Q}")
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = counter_harmonic_mean_filter(images_noised[image_name], kernel_size=kernel_size, Q=Q)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

Q = 1.0


In [2304]:
Q = 1.25
# Применеие Контргармонического усредняющего фильтра с различными значениями параметра 𝑄 
# ко всем зашумлённым изображениям
print(f"Q = {Q}")
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = counter_harmonic_mean_filter(images_noised[image_name], kernel_size=kernel_size, Q=Q)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

Q = 1.25


In [2305]:
Q = 1.5
# Применеие Контргармонического усредняющего фильтра с различными значениями параметра 𝑄 
# ко всем зашумлённым изображениям
print(f"Q = {Q}")
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = counter_harmonic_mean_filter(images_noised[image_name], kernel_size=kernel_size, Q=Q)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

Q = 1.5


### 3 Нелинейная фильтрация.

Низкочастотные фильтры линейны и оптимальны в случае, когда имеет место нормальное распределение помех на цифровом
изображении. Линейные фильтры локально усредняют импульсные
помехи, сглаживая изображения. Для устранения импульсных помех лучше использовать нелинейные, например, медианные фильтры.

### 3.1 Медианный фильтр

In [2306]:
kernel = np.ones((11,11),np.float64)
print(kernel)
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = scipy.ndimage.median_filter(images_noised[image_name], footprint=kernel)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


### 3.2 Взвешенный медианный фильтр

In [2307]:
kernel = np.random.random(size=(11,11))
kernel /= np.max(kernel)
print(kernel)
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = scipy.ndimage.median_filter(images_noised[image_name], footprint=kernel)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

[[0.64615637 0.26977317 0.7749899  0.56432424 0.47330154 0.45347919
  0.26698036 0.4790399  0.26517087 0.75194751 0.68216014]
 [0.26085918 0.85150096 0.32871482 0.20020641 0.33565366 0.38220901
  0.1470468  0.9037262  0.19183328 0.86404635 0.54162431]
 [0.20642388 0.97068014 0.9702673  0.76154346 0.54065638 0.1414458
  0.23206408 0.94983336 0.26998171 0.59466407 0.81595462]
 [0.29173326 0.89623383 0.13177004 0.19992196 0.63610602 0.57974774
  0.33252451 1.         0.92632545 0.1733901  0.33820014]
 [0.88891459 0.07538242 0.85431183 0.736838   0.87911273 0.40698161
  0.23919947 0.1592842  0.21949358 0.39685837 0.44467035]
 [0.21141293 0.57103085 0.1923683  0.76570725 0.39507131 0.26014308
  0.57052446 0.6553527  0.11486183 0.77337701 0.08649739]
 [0.62964108 0.50512294 0.1604947  0.02084639 0.59740375 0.87129288
  0.79912102 0.2819659  0.08016646 0.24369918 0.3384817 ]
 [0.24876532 0.51251902 0.08373699 0.50007221 0.54249013 0.96438031
  0.58058421 0.94884031 0.69452861 0.32479593 0.393

### 3.3 Ранговый фильтр

In [2308]:
rank = -25
kernel = np.ones((5,5),np.float64)
print(f"rank = {rank + 25}")
print(kernel)
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = scipy.ndimage.rank_filter(images_noised[image_name], footprint=kernel, rank=rank)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

rank = 0
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [2309]:
rank = -18
kernel = np.ones((5,5),np.float64)
print(f"rank = {rank + 25}")
print(kernel)
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = scipy.ndimage.rank_filter(images_noised[image_name], footprint=kernel, rank=rank)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

rank = 7
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [2310]:
rank = -12
kernel = np.ones((5,5),np.float64)
print(f"rank = {rank + 25}")
print(kernel)
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = scipy.ndimage.rank_filter(images_noised[image_name], footprint=kernel, rank=rank)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

rank = 13
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [2311]:
rank = -6
kernel = np.ones((5,5),np.float64)
print(f"rank = {rank + 25}")
print(kernel)
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = scipy.ndimage.rank_filter(images_noised[image_name], footprint=kernel, rank=rank)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

rank = 19
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [2312]:
rank = -1
kernel = np.ones((5,5),np.float64)
print(f"rank = {rank + 25}")
print(kernel)
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = scipy.ndimage.rank_filter(images_noised[image_name], footprint=kernel, rank=rank)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

rank = 24
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


### 3.4 Фильтр Винера

Также добавлено адаптивное автоисправление гистограммы CLAHE

In [2313]:
kernel = np.ones((15,15),np.float64)
print(kernel)
if use_graph:
    image_combined = None
    for image_name in images_noised.keys():
        print(image_name, end=', ')
        image_filtered = skimage.img_as_float64(images_noised[image_name])
        image_filtered = scipy.signal.convolve2d(image_filtered, kernel, 'same')
        image_filtered = skimage.restoration.wiener(image_filtered, kernel, 5.1e4)
        image_filtered = skimage.img_as_ubyte(image_filtered)
        clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(8,8))
        image_filtered = clahe.apply(image_filtered)
        image_filtered = cv2.hconcat([images_noised[image_name], image_filtered])
        if image_combined is None:
            image_combined = image_filtered
        else:
            image_combined = cv2.vconcat([image_combined, image_filtered])
    print()
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_combined, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [2314]:
kernel = np.ones((15,15),np.float64)
print(kernel)
if use_graph:
    image_filtered = skimage.img_as_float64(image_noised_all)
    image_filtered = scipy.signal.convolve2d(image_filtered, kernel, 'same')
    image_filtered = skimage.restoration.wiener(image_filtered, kernel, 1.3e6)
    image_filtered = skimage.img_as_ubyte(image_filtered)
    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(8,8))
    image_filtered = clahe.apply(image_filtered)
    image_filtered = cv2.hconcat([image_noised_all, image_filtered])
    figure(figsize=(32, 16), dpi=80)
    plt.imshow(cv2.cvtColor(image_filtered, cv2.COLOR_GRAY2RGB))
    plt.xticks([]),plt.yticks([])
    plt.show()

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
